# Brief Summary of The Classifiers In The Outline

In terms of accuracy, the decision tree classifier is the most accurate followed by the MiniRocket classifier then followed by the ResNet50 convolutional neural network which was applied to the chi square maps. Both the MiniRocket and ResNet50 do not require feature engineering from the user prior to their use which is an advantage they hold over the decision tree.

The decision tree and MiniRocket in particular consistently have accuracies over 90% despite the smaller sample size of the data used to train them in comparison to the data available. The reason for the use of a smaller sample size is due to a need to balance the data between sources classified as transients and non-transients to minimise bias within the classifiers.

Additionally, MiniRocket and ResNet50 have been capable of predicting outliers which the decision tree cannot which is another advantage they hold.

To cover as many likely transient candidates, this pipeline obtains predicted transients from each classifier and groups them into 1 large dataframe. Limitations in the pipeline aside from the natural limitations of each classifier pertain to the fact that the light curves, fits cubes and feature data do not have a direct correspondence between each other. For instance there may be sources in the feature data for which we are missing light curves or fits cubes so we are unable to classify those sources in one or more of the classifiers.

Nonetheless, these 3 different classifiers should aid in reducing the amount of sources that a researcher of such transient phenomena would need to examine.

# Import The Relevant Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [2]:
# This keeps the seed for keras and tensorflow consistent across scripts
from numpy.random import seed 
# keras seed fixing import
seed(1)

# tensorflow seed fixing
tf.random.set_seed(1)

In [3]:
import pickle
import pandas as pd
from astropy.io import fits

from tensorflow.keras.utils import to_categorical

from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import recall_score

from sktime.utils import mlflow_sktime

# Obtain and Preprocess The Data For Each Classifier

In [4]:
# Change data file name to whichever data file you are using
data_file_name = "All_Transient_Data.csv"

# Folder names
lc_folder_name = "VAST 10s lightcurve"
fits_folder_name = "VAST 10s fitscube"

In [5]:
t_data = pd.read_csv(data_file_name)

In [6]:
# Create a subset of the transient data with the light curve paths and the fits file paths
rel_data = t_data[["sbid","beam","name"]]
rel_data = rel_data.dropna(how='any',axis=0)
rel_data["sbid"] = rel_data["sbid"].astype(str)
rel_data["lc_peakflux_path"] = "SB"+rel_data["sbid"]+"_"+rel_data["beam"]+"_lightcurve_peak_flux.csv"

rel_data["fits_path"] = "SB"+rel_data["sbid"]+"_"+rel_data["beam"]+"_slices_"+rel_data["name"]+".fits"

all_lc_files = os.listdir(lc_folder_name)

# Collect the light curve paths in the data subset that actually exist
match_files = []
for i in list(rel_data["lc_peakflux_path"].unique()):
    if i in all_lc_files:
        match_files.append(i)

### Preprocessing For Decision Tree

In [7]:
def process_data(df, feature_cols):
    """
    Replaces infinite values with 1,000,000 and replaces NaN values with the column's median.
    This is only done for particular columns of interest.

    Parameters
    ----------
    df : pandas dataframe
        Dataframe we want to process

    feature_cols : list
        List of strings og features or columns in the data that we want to process.

    Returns
    -------
    df : pandas dataframe
        The processed dataframe.
    """
    # Replace inf values with large floats so the classifier can use the value
    # Choice of 1E6 because it is much larger than the maximum values of columns with inf as a value.
    df[feature_cols] = df[feature_cols].replace(np.inf, float(1E6))

    # Replace NaN values with the median
    df[feature_cols] = df[feature_cols].fillna(df[feature_cols].median())
    return df

In [8]:
# process data for decision tree
feature_cols = ['chi_square', 'chi_square_sigma', 'peak_map', 'peak_map_sigma','md_deep', 'deep_sep_arcsec', 
                'deep_num', 'bright_sep_arcmin', 'beam_sep_deg', 'deep_peak_flux', 'deep_int_flux','std_map']
dt_data = process_data(t_data, feature_cols)

### Preprocessing For MiniRocket

In [9]:
def normalise_signal(signal):
    """
    Applies min-max normalisation on an array or signal.

    Parameters
    ----------
    signal : list or numpy array
        A list or array of integers or floats.

    Returns
    -------
    signal : numpy array
        A min-max normalised numpy array
    """
    # Min-max normalisation so values are in between 0 and 1
    signal = np.array(signal)
    signal = (signal - signal.min())/ (signal.max() - signal.min())
    return signal

In [10]:
def obtain_data_seq(file_name, rel_data, lc_folder_name, min_seq_size = 64):
    """
    Obtains the truncated sequences and source information for each source
    in a light curve file.

    Parameters
    ----------
    file_name : str
        The name of the file with it's extension.

    rel_data : pandas dataframe
        The dataframe containing the relevant features and
        associated peak flux light curve paths.

    lc_folder_name : str, "VAST 10s lightcurve"
        The path to the light curve folder

    min_seq_size : int, default = 64
        The minimum length of the sequences which restricts how large
        the other sequences can be.

    Returns
    -------
    sequences : list
        List of all the obtained light curves.

    src_names : list
        List of all the obtained source information for the light curves.
    """
    pf_df = pd.read_csv(f"{lc_folder_name}/{file_name}")
    
    split_file = file_name.split("_")
    sbid = split_file[0][2:]
    beam_id = split_file[1]
    sequences = []
    src_names = []
    for src in pf_df.columns[1:]:
        # Need to denoise the signals
        val = pf_df[src].values
        
        # Need to normalise row values
        val = normalise_signal(val)
        # truncate the rows so all rows are the same size
        val = val[:min_seq_size]
        val = list(val)
        src_names.append(f"{sbid}_{beam_id}_{src}")
        sequences.append(val)

    return sequences, src_names

In [11]:
# Obtain the sequences and source information from the light curve file
all_sequences = []
all_src_names = []
for file in match_files:
    sequences, src_names =  obtain_data_seq(file, rel_data, lc_folder_name, min_seq_size = 64)
    all_sequences.extend(sequences)
    all_src_names.extend(src_names)

In [12]:
seq_array = np.array(all_sequences)
src_array = np.array(all_src_names)

In [13]:
# Remove missing values
src_array = src_array[~np.isnan(seq_array).any(axis=1)]
seq_array = seq_array[~np.isnan(seq_array).any(axis=1),:]

In [14]:
# Make sequences a numpy 3D array for the rocket classifier
seq_array = seq_array[:, np.newaxis, :]

In [15]:
seq_array.shape

(2525, 1, 64)

### Preprocessing for ResNet

In [16]:
def obtain_avail_fits(rel_data, fits_folder_name):
    """
    Obtains the fits files which are in both the given dataframe
    and the fits folder.

    Parameters
    ----------
    rel_data : pandas dataframe
        The dataframe with all the possible fits paths.

    fits_folder_name : string, default "VAST 10s fitscube"
        The fits folder path.
    
    Returns
    -------
    avail_fits : list
        A list of strings of the selected fits file names.
    """
    # List of files in fits_folder and the actual data
    avail_fits = []

    for file in os.listdir(fits_folder_name):
        if file in rel_data["fits_path"].unique():
            avail_fits.append(file)

    return avail_fits

In [17]:
def obtain_rms(fits_path, lc_folder_name):
    """
    Obtains the local root mean squared data for a particular fits file.

    Parameters
    ----------
    fits_path : str
        The path for a fits file.

    lc_folder_name : str
        The path for a light curve folder.

    Returns
    -------
    rms : numpy array
        An array containing the local root mean squared values for a particular array.
    """
    # Need to return indices which have rms = 0 or nan
    sbid, beam, _, name = fits_path.split("_")
    name = name[:-5]
    rms_path = f"{sbid}_{beam}_lightcurve_local_rms.csv"

    # Need to ensure that a light curve exists 
    if rms_path not in os.listdir(lc_folder_name):
        return np.array([])
    else:
        local_rms_df = pd.read_csv(f"{lc_folder_name}//{rms_path}")
        rms = local_rms_df[name]
        rms = np.array(rms)
        rms = rms[:, np.newaxis, np.newaxis]
        return rms

In [18]:
def obtain_cube(fits_path, fits_folder_name):
    """
    Obtains a 3D fits cube with values relating to images taken at different
    time points during an observation.

    Parameters
    ----------
    fits_path : str
        The path for a fits file.

    lc_folder_name : str
        The path for a light curve folder.

    Returns
    -------
    cube : numpy array
        A 3D fits cube corresponding to the given fits file.
    """
    # Only take slices which do not have a corresponding rms equal to 0
    hdu = fits.open(fits_folder_name+"/"+fits_path)
    cube = hdu[0].data

    return cube

In [19]:
def obtain_csm(cube, rms):
    """
    Obstains the chi square maps for
    a particular fits cube.

    Parameters
    ----------
    cube : numpy array
        A fits cube containing images of a particular observation
        at different time points.

    rms : numpy array
        The local rms of the associated cube.

    Returns
    -------
    chisq_array : numpy array
        The chi square map for the input fits cube.
    """
    # number of freedom
    nu = cube.shape[0] - 1
    mean = np.nanmean(cube, axis=0)
    
    data = (cube - mean) / rms
    chisq_array = np.sum(np.power(data, 2), axis=0)/nu

    # normalise the array
    chisq_array = normalise_signal(chisq_array)
    chisq_array = np.repeat(chisq_array[...,np.newaxis], 3, -1)
    
    return chisq_array

In [20]:
# Finding the available fits files
avail_fits = obtain_avail_fits(rel_data, fits_folder_name)

In [21]:
# Filter the available fits paths such that only those which have a valid local rms file are obtained
adj_avail_fits = []
for i, fits_path in enumerate(avail_fits):
        rms = obtain_rms(fits_path, lc_folder_name)
        # if there is no corresponding rms curve for a fits file, skip this iteration
        if rms.size == 0:
            continue

        # Only if the rms values do not contain nan values will I obtain the cube
        if not np.isnan(rms).any():
            adj_avail_fits.append(avail_fits[i])

adj_avail_fits = np.array(adj_avail_fits)

In [22]:
# Filter fits files based on valid cube data and peak maps which pass a threshold
map_size = (120,120)

# Indices where the cube's shape is not what we expect from the map size
invalid_idx = []

for i, fits_path in enumerate(adj_avail_fits):
        rms = obtain_rms(fits_path, lc_folder_name)
        # Take the first non-zero value onwards
        first_non_zero_idx = np.argmax(rms != 0)
        rms = rms[first_non_zero_idx:,:,:]
        cube = obtain_cube(fits_path, fits_folder_name)
        cube = cube[first_non_zero_idx:,:,:]
    
        if (cube.shape[1:] != map_size):
            invalid_idx.append(i)

In [23]:
adj_avail_fits = np.delete(adj_avail_fits, invalid_idx)

In [24]:
# Obtain all the chi-square maps

num_channels = 3

# Initialise a numpy array to store the chi-square maps
all_csm = np.zeros((len(adj_avail_fits), map_size[0], map_size[1], num_channels))
for i, fits_path in enumerate(adj_avail_fits):
    rms = obtain_rms(fits_path, lc_folder_name)
    # Take the first non-zero value onwards
    first_non_zero_idx = np.argmax(rms != 0)
    rms = rms[first_non_zero_idx:,:,:]
    cube = obtain_cube(fits_path, fits_folder_name)
    cube = cube[first_non_zero_idx:,:,:]
    # Obtain the chi squared maps
    csm = obtain_csm(cube, rms)
    all_csm[i,:,:,:] = csm

# *Decision Tree Predictions*

In [25]:
# Load the decision tree classifier
tree_path = "Models/Tree_clf/Tree.pkl"
with open(tree_path, "rb") as f:
    tree_clf = pickle.load(f)

In [26]:
dt_data.head()

,source_id,name,ra_str,dec_str,ra,dec,chi_square,chi_square_log_sigma,chi_square_sigma,peak_map,...,beam,sbid,PSR_name,PSR_sep,dyspec,KNOWN_name,KNOWN_sep,PSR_name_int,KNOWN_name_int,PSR_Label
0,0,J163259.92-501507.22,16:32:59.92,-50:15:07.22,248.249681,-50.252004,2.874275,6.047280,7.511542,4.286386,...,beam00,49588,J1633-5015,1.934309,NaN,NaN,NaN,1,0,1
1,1,J163048.20-491129.49,16:30:48.20,-49:11:29.49,247.700817,-49.191524,2.465644,5.211199,6.240665,3.523429,...,beam00,49588,NaN,NaN,NaN,NaN,NaN,0,0,0
2,0,J162710.82-481537.04,16:27:10.82,-48:15:37.04,246.795077,-48.260289,3.333277,6.090706,8.813457,3.578227,...,beam04,49588,NaN,NaN,NaN,NaN,NaN,0,0,0
3,0,J163250.42-482506.53,16:32:50.42,-48:25:06.53,248.210091,-48.418480,3.578913,5.044124,9.465821,4.424191,...,beam05,49588,NaN,NaN,NaN,NaN,NaN,0,0,0
4,0,J164019.07-490047.32,16:40:19.07,-49:00:47.32,250.079454,-49.013143,3.547815,5.492090,9.384754,4.350268,...,beam06,49588,NaN,NaN,NaN,NaN,NaN,0,0,0


In [27]:
# Use the classifier for predictions on the new data
tree_preds = tree_clf.predict(dt_data[feature_cols])

In [28]:
tree_preds.shape

(7714,)

In [29]:
np.unique(tree_preds, return_counts = True)

(array([0, 1], dtype=int64), array([7080,  634], dtype=int64))

In [30]:
# Create dataframe from source names, sbid and beam id.
tree_cand_df = dt_data.iloc[tree_preds == 1][['sbid','beam','name']]

In [31]:
tree_cand_df.head()

,sbid,beam,name
0,49588,beam00,J163259.92-501507.22
13,49588,beam17,J163037.34-473302.59
15,49589,beam06,J165148.70-424609.76
16,49589,beam06,J164946.18-423017.80
17,49589,beam07,J165148.76-424611.17


# *MiniRocket Predictions*

In [32]:
# Load the minirocket classifier
rocket_path = f"Models/Rocket_Model"
rocket_clf = mlflow_sktime.load_model(model_uri=rocket_path)

In [33]:
# Use the classifier for predictions on the new sequences
rocket_preds = rocket_clf.predict(seq_array)

In [34]:
np.unique(rocket_preds, return_counts = True)

(array([0, 1]), array([2060,  465], dtype=int64))

In [35]:
# Obtain srcs
rocket_cand_srcs = src_array[rocket_preds == 1]

In [36]:
# Obtain the sbid, beam and names of the sources
rocket_sbid = []
rocket_beam = []
rocket_name = []
for src in rocket_cand_srcs:
    sbid, beam, name = src.split("_")
    rocket_sbid.append(int(sbid))
    rocket_beam.append(beam)
    rocket_name.append(name)

In [37]:
# Create a dictionary
rocket_cand_dict = {"sbid":rocket_sbid, "beam":rocket_beam, "name":rocket_name}
rocket_cand_df = pd.DataFrame(rocket_cand_dict)

In [38]:
rocket_cand_df.head()

,sbid,beam,name
0,60802,beam15,J174556.28-304022.46
1,60802,beam19,J175258.66-280637.15
2,60803,beam08,J174802.33-244636.78
3,60803,beam17,J173326.43-222845.82
4,60803,beam26,J175258.73-280636.72


# *Chi-Square Map ResNet Predictions*

In [39]:
# Load the best hyper parameters in a dictionary
fpath = "Models/CSM_Resnet"
with open(f"{fpath}/hyperparams.pkl", "rb") as f:
    hp_dict = pickle.load(f)

n_classes = 2

In [40]:
resnet_model = keras.Sequential()
        
input_shape = (map_size[0], map_size[1], num_channels)
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=input_shape,
                   pooling='avg',classes=n_classes,
                   weights='imagenet')

# Don't train the pretrained layers
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(units=hp_dict['units'], activation='relu'))
# Choose sigmoid activation for binary classification
resnet_model.add(Dense(n_classes, activation='sigmoid'))
#resnet_model.build((None, input_shape[0], input_shape[1], input_shape[2]))

# Choose binary crossentropy loss function for binary classification
resnet_model.compile(optimizer=Adam(learning_rate = hp_dict['learning_rate']),loss='binary_crossentropy',metrics=['binary_accuracy'])

In [41]:
# Test on all_csm
resnet_model.load_weights(f"{fpath}/weights.h5")

In [42]:
resnet_preds = resnet_model.predict(all_csm)

78/78 [==============================] - 6s 15ms/step


In [43]:
resnet_preds_decoded = []
for pred in resnet_preds:
    resnet_preds_decoded.append(np.argmax(pred))
resnet_preds_decoded = np.array(resnet_preds_decoded)

In [44]:
# Determine how many of each class is predicted
np.unique(resnet_preds_decoded, return_counts = True)

(array([0, 1], dtype=int64), array([2021,  461], dtype=int64))

In [45]:
# Obtain relevant src_info
cand_fits_files = adj_avail_fits[resnet_preds_decoded == 1]
resnet_sbid = []
resnet_beam = []
resnet_name = []
for file in cand_fits_files:
    sbid, beam, _, name = file.split("_")
    resnet_sbid.append(int(sbid[2:]))
    resnet_beam.append(beam)
    resnet_name.append(name[:-5])

In [46]:
# create a dataframe from these values
resnet_cand_dict = {"sbid":resnet_sbid, "beam":resnet_beam, "name": resnet_name}
resnet_cand_df = pd.DataFrame(resnet_cand_dict)

In [47]:
resnet_cand_df.head()

,sbid,beam,name
0,60802,beam15,J174556.28-304022.46
1,60802,beam19,J175258.66-280637.15
2,60803,beam08,J174802.33-244636.78
3,60803,beam17,J173326.43-222845.82
4,60803,beam26,J175258.73-280636.72


### Issue With The Loaded ResNet Model

When the saved ResNet model from the 'Chi_Square_Map_and_Peak_Map_ResNet' notebook is loaded in a new notebook, the predictions on the same set of data may differ each time this notebook is run. Attempted solutions included setting the accuracy metric in the ResNet architecture of both notebooks to 'binary_accuracy', setting a seed for the keras and tensorflow, stopping the random shuffling and saving the entire model and loading it in this notebook.

None of these methods have worked in addressing this issue.

# Collect All The Candidates

In [48]:
# Combine the candidates into 1 dataframe.
cand_df = pd.concat([tree_cand_df, rocket_cand_df, resnet_cand_df], ignore_index=True, axis=0)

In [49]:
# Drop all the duplicate rows
cand_df.drop_duplicates(inplace = True, ignore_index = True)

In [50]:
cand_df.head()

,sbid,beam,name
0,49588,beam00,J163259.92-501507.22
1,49588,beam17,J163037.34-473302.59
2,49589,beam06,J165148.70-424609.76
3,49589,beam06,J164946.18-423017.80
4,49589,beam07,J165148.76-424611.17


In [51]:
# Save the candidates
cand_fol = "Transient_candidates"
cand_df.to_csv(f"{cand_fol}/transient_candidates.csv")